In [1]:
# !pip install langchain chromadb faiss-cpu pypdf transformers sentence_transformers

In [2]:
!pip install -U langchain langchain-community faiss-cpu chromadb pypdf transformers sentence-transformers

In [15]:
!pip install -U :class:`~langchain-huggingface

/bin/bash: -c: line 1: unexpected EOF while looking for matching ``'
/bin/bash: -c: line 2: syntax error: unexpected end of file


In [34]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.llms import HuggingFacePipeline
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from transformers import pipeline

In [35]:
pdf_path = '/content/AI2.pdf'
loader = PyPDFLoader(pdf_path)
docs = loader.load()
print(docs[0].page_content)

In [36]:
docs

[Document(metadata={'producer': 'Acrobat Distiller 5.0 (Windows)', 'creator': 'PyPDF', 'creationdate': '2007-05-24T11:04:58+05:30', 'moddate': '2007-05-24T11:05:18+05:30', 'title': 'Artificial Intelligence Illuminated', 'author': 'Coppin, Ben.', 'source': '/content/AI2.pdf', 'total_pages': 768, 'page': 0, 'page_label': '1'}, page_content=''),
 Document(metadata={'producer': 'Acrobat Distiller 5.0 (Windows)', 'creator': 'PyPDF', 'creationdate': '2007-05-24T11:04:58+05:30', 'moddate': '2007-05-24T11:05:18+05:30', 'title': 'Artificial Intelligence Illuminated', 'author': 'Coppin, Ben.', 'source': '/content/AI2.pdf', 'total_pages': 768, 'page': 1, 'page_label': '2'}, page_content='Artificial Intelligence \nIlluminated\nBen Coppin\nJONES AND BARTLETT PUBLISHERS'),
 Document(metadata={'producer': 'Acrobat Distiller 5.0 (Windows)', 'creator': 'PyPDF', 'creationdate': '2007-05-24T11:04:58+05:30', 'moddate': '2007-05-24T11:05:18+05:30', 'title': 'Artificial Intelligence Illuminated', 'author': 

In [37]:
text = [doc.page_content for doc in docs]
len(text)

768

In [38]:
splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 50)
docs = splitter.create_documents(text)
print(docs)

[Document(metadata={}, page_content='Artificial Intelligence \nIlluminated\nBen Coppin\nJONES AND BARTLETT PUBLISHERS'), Document(metadata={}, page_content='Copyright © 2004 by Jones and Bartlett Publishers, Inc.\nCover image © Photodisc\nLibrary of Congress Cataloging-in-Publication Data\nCoppin, Ben.\nArtificial intelligence illuminated / by Ben Coppin.--1\nst ed.\np. cm.\nIncludes bibliographical references and index.\nISBN 0-7637-3230-3\n1. Artificial intelligence. I. Title.\nQ335.C586  2004\n006.3--dc22\n2003020604\n3382\nAll rights reserved. No part of the material protected by this copyright notice may be'), Document(metadata={}, page_content='reproduced or utilized in any form, electronic or mechanical, including photocopying,\nrecording, or any information storage or retrieval system, without written permission\nfrom the copyright owner.\nAcquisitions Editor: Stephen Solomon\nProduction Manager: Amy Rose\nMarketing Manager: Matthew Bennett\nEditorial Assistant: Caroline Senay\

In [39]:
embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')

In [40]:
db = FAISS.from_documents(
    documents = docs,
    embedding = embeddings
)

In [41]:
results = db.similarity_search_with_score("what is Ai?", k=2)
for res, score in results:
    print(f"{score:.3f} → {res.page_content}")

0.527 → AI. see Artificial intelligence (AI)
AI: Artificial Intelligence (film), 23
AIBO robotic dog, 23
Alan Turing the Enigma of Intelligence(Hodge), 291
Alfonso the Wise, 117
Algorithms
acquaintance, 633
AD3, 268, 278, 281, 661
bucket brigade, 286, 377, 378, 379–380, 639
genetic. See Genetic algorithms
GraphPlan, 434, 451, 454–455, 658
winner-take-all, 316, 694
0.671 → 1.11 Chapter Summary
■ Intelligence is difficult to define, and as a result Artificial Intelli-
gence is also hard to define.
■ One definition of Artificial Intelligence is:
Artificial intelligence is the study of systems that act in a way that to
any observer would appear to be intelligent.
■ Proponents of strong AI believe that a computer that behaves in an
intelligent way is capable of possessing mental states and, there-
fore, of being truly conscious and intelligent in the same way that


In [42]:
retriver = db.as_retriever()

In [43]:
generator = pipeline(
    'text2text-generation',
    model = 'facebook/bart-large-cnn'
)

Device set to use cpu


In [44]:
llm = HuggingFacePipeline(pipeline = generator)
qa_chain = RetrievalQA.from_chain_type(
    llm = llm,
    retriever = retriver
)

In [45]:
def answer_question(question):
    try:
        return qa_chain.run(question)
    except Exception as e:
        return str(e)

In [46]:
answer_question("Fields of Ai?")

"The first few chapters of this book provide introductory material. Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don’t know, don't try to make up an answer. In studying Artificial Intelligence, it is extremely useful to understand the background of philosophy, linguistics, biology, and psychology."

In [47]:
print(f"Answer: {answer_question('define Neural Networks')}")

Answer: An artificial neural network is a network of simple processing nodes, roughly modeled on the human brain. The word artificial is often used to describe neural networks, but in this book we shall simply refer to them as neural networks because it should be clear from the context which type of network we are referring to. Use the following pieces of context to answer the question at the end.


In [49]:
while True:
    question = input("Enter your question (or 'exit' to quit): ")
    if question.lower() == 'exit':
        break
    answer = answer_question(question)
    print(f"Answer: {answer}\n\n")

Enter your question (or 'exit' to quit): Who coined the term Artificial Intelligence and when?
Answer: In 1956, the term Artificial Intelligence was first used by John McCarthy at Dartmouth College, in Hanover, New Hampshire. In 1957, Newell and Simon invented the idea of the GPS, whose purpose was to solve almost any logical problem. In studying Artificial Intelligence, it is extremely useful to understand the background of philosophy, linguistics, biology, and psychology.


Enter your question (or 'exit' to quit): What is the difference between strong AI and weak AI?
Answer: In this chapter, we will look at the contributions made by philosophy, lin-                guistics, psychology, and biology to Artificial Intelligence. We will also look. at the difference between the claims made by proponents of weak AI (AI is. a commonly used abbreviation for Artificial Intelligence) compared with those who support strong AI.


Enter your question (or 'exit' to quit): Which programming languag

In [50]:
import gradio as gr
import requests

iface = gr.Interface(
    fn=answer_question,
    inputs=gr.Textbox(lines=2, placeholder="Type your question here... 🤔"),
    outputs="text",
    title="🌐 Web QA System",
    description="Ask any question about the selected websites and get answers from the extracted information! 📝"
)

iface.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5940b0525d59ed6ed9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
